# Importing Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from DataSet import DataSet as DS
%matplotlib qt

# Building A Mesh and Loading $\vec{B}$
I used one of Daniel's data (output of MFM simulation)

In [2]:
BaseAddress = "/home/farhadda/BackUp/"
ds1 = DS(SystemOfCoords="CAR", BaseAddress=BaseAddress,
         Pattern="div_E_omega_2017-09-06T11-24-00.npz",
         NBlocks=1, UseBlock=0)
print("Number of cells in each direction : ", ds1.NCell)

---<( Input files include Ghost-Cells, those cells will be ignored )>---
Following parameters are found in the given file :: 
 Bx (defined on FaceX)
 By (defined on FaceY)
 Bz (defined on FaceZ)
Bx	is loaded successfully.
By	is loaded successfully.
Bz	is loaded successfully.
Number of cells in each direction :  (249, 192, 229)


# Check Div. B for input Magnetic Field

In [3]:
ds1.DivFace("Bx", "By", "Bz", "DivB")
a = np.abs(ds1.vars["DivB"]['val'])
print(a.mean(), a.max(), a.min(), sep='\t')
print(*np.where(a == a.max()))

4.460892352578051e-05	0.2916412353515625	0.0
[108] [98] [0]


# Notice :
It seems that input magnetic field __is not  Div. Free__

In [4]:
ds1.ExtractAFromBFace("Bx", "By", "Bz", "Ax", "Ay", "Az")
ds1.CurlEdgeToFace("Ax", "Ay", "Az", "Bx2", "By2", "Bz2")

plt.figure()
plt.imshow((ds1.vars["Bx"]['val'][:, :, 0] -
            ds1.vars["Bx2"]['val'][:, :, 0]).T, origin="Lower Left")
plt.tight_layout()
plt.title(r"Diff $B_{x}$ on Bottom")
plt.colorbar()

plt.figure()
plt.imshow((ds1.vars["By"]['val'][:, :, 0] -
            ds1.vars["By2"]['val'][:, :, 0]).T, origin="Lower Left")
plt.tight_layout()
plt.title(r"Diff $B_{y}$ on Bottom")
plt.colorbar()

plt.figure()
plt.imshow((ds1.vars["Bz"]['val'][:, :, 0] -
            ds1.vars["Bz2"]['val'][:, :, 0]).T, origin="Lower Left")
plt.tight_layout()
plt.title(r"Diff $B_{z}$ on Bottom")
plt.colorbar()

In [11]:
ds1.Write2HDF5(BaseAddress+"InitialMesh.h5")

In [13]:
x0 = ds1.nodevect["X"][0]
x1 = ds1.nodevect["X"][-1]
y0 = ds1.nodevect["Y"][0]
y1 = ds1.nodevect["Y"][-1]
z0 = ds1.nodevect["Z"][0]
z1 = ds1.nodevect["Z"][-1]
ds2 = DS(SystemOfCoords="CAR", NCell=ds1.NCell,
         startval=(x0, y0, z0), endval=(x1, y1, z1))

In [15]:
ds1.ToNewMesh(ds2, "Ax", "EdgeX")
ds1.ToNewMesh(ds2, "Ay", "EdgeY")
ds2.Scalar("Az", "EdgeZ", ds2.Adummy)

In [16]:
ds2.CurlEdgeToFace("Ax", "Ay", "Az", "Bx", "By", "Bz")
ds2.DivFace("Bx", "By", "Bz", "DivB")

In [17]:
ds2.Write2HDF5(BaseAddress+"SecondMesh.h5")

In [18]:
del ds2

In [7]:
ds3 = DS(SystemOfCoords="CAR", NCell=(200, 180, 200),
         startval=(x0, y0, z0), endval=(x1, y1, z1))

In [8]:
ds1.ToNewMesh(ds3, "Ax", "EdgeX")
ds1.ToNewMesh(ds3, "Ay", "EdgeY")
ds3.Scalar("Az", "EdgeZ", ds3.Adummy)

In [9]:
ds3.CurlEdgeToFace("Ax", "Ay", "Az", "Bx", "By", "Bz")
ds3.DivFace("Bx", "By", "Bz", "DivB")

In [10]:
ds3.Write2HDF5(BaseAddress+"ThirdMesh.h5")